In [1]:
from prettytable import PrettyTable as PT

# <b>Pairwise 1 Clusters Count kCluster->CDH </b>

In [2]:
empty_d = {"CC":0, "IC":0, "CM":0, "IM":0}
kCl_to_CDH = {"CC":dict(empty_d), "IC":dict(empty_d), "CM":dict(empty_d), "IM":dict(empty_d)}
with open("uniq_bio_assess.tsv", 'r') as tsv:
    next(tsv)
    for line in tsv:
        line = line.split()
        kCl_ID = line[0]
        kCl_type = line[1]
        CDH_ID = line[2]
        CDH_type = line[3]
        kCl_to_CDH[kCl_type][CDH_type] += 1

pr = PT()
pr.field_names = ["kCl/CDH","CC","IC","CM","IM"]
total_clusters = 0
TYPES = ["CC","IC","CM","IM"]
for TYPE in TYPES:
    row = []
    for T in TYPES:
        total_clusters += kCl_to_CDH[TYPE][T]
        row.append(kCl_to_CDH[TYPE][T])
    pr.add_row([TYPE]+row)
    
print pr
print "Total Clusters: ", total_clusters

+---------+------+-------+-----+-----+
| kCl/CDH |  CC  |   IC  |  CM |  IM |
+---------+------+-------+-----+-----+
|    CC   | 9287 | 10579 | 289 | 515 |
|    IC   | 505  | 13509 |  9  | 212 |
|    CM   |  6   |   64  | 110 | 135 |
|    IM   |  1   |   47  |  7  | 124 |
+---------+------+-------+-----+-----+
Total Clusters:  35399


# <b>Pairwise 2 Clusters Count CDH->kCluster </b>

In [3]:
empty_d = {"CC":0, "IC":0, "CM":0, "IM":0}
CDH_to_kCl = {"CC":dict(empty_d), "IC":dict(empty_d), "CM":dict(empty_d), "IM":dict(empty_d)}
with open("uniq_bio_assess.tsv", 'r') as tsv:
    next(tsv)
    for line in tsv:
        line = line.split()
        kCl_ID = line[0]
        kCl_type = line[1]
        CDH_ID = line[2]
        CDH_type = line[3]
        CDH_to_kCl[CDH_type][kCl_type] += 1

pr2 = PT()
pr2.field_names = ["CDH/kCl","CC","IC","CM","IM"]
total_clusters = 0
TYPES = ["CC","IC","CM","IM"]
for TYPE in TYPES:
    row = []
    for T in TYPES:
        total_clusters += CDH_to_kCl[TYPE][T]
        row.append(CDH_to_kCl[TYPE][T])
    pr2.add_row([TYPE]+row)
    
print pr2
print "Total Clusters: ", total_clusters

+---------+-------+-------+-----+-----+
| CDH/kCl |   CC  |   IC  |  CM |  IM |
+---------+-------+-------+-----+-----+
|    CC   |  9287 |  505  |  6  |  1  |
|    IC   | 10579 | 13509 |  64 |  47 |
|    CM   |  289  |   9   | 110 |  7  |
|    IM   |  515  |  212  | 135 | 124 |
+---------+-------+-------+-----+-----+
Total Clusters:  35399
